 ## Transformation des données

Nettoyage : suppression ou traitement des valeurs manquantes et incohérentes.
Standardisation : harmonisation des unités, formats de date et noms de colonnes pour garantir la cohérence et la qualité des données.

In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

# Configuration des chemins
BRONZE_PATH = "../../data/bronze"
SILVER_PATH = "../../data/silver"

# Créer le dossier silver
os.makedirs(SILVER_PATH, exist_ok=True)

# ============================================================================
# FONCTIONS DE NETTOYAGE
# ============================================================================

def clean_numeric_column(series):
    """Nettoie et convertit une colonne en numérique"""
    series = series.replace(['', '-', ' ', 'N/A'], np.nan)
    series = pd.to_numeric(series, errors='coerce')
    series = series.fillna(0)
    return series

def clean_age(age_str):
    """Nettoie la colonne Age (format: '25-123' ou '25')"""
    if pd.isna(age_str) or age_str == '':
        return np.nan
    age = str(age_str).split('-')[0].strip()
    try:
        return int(age)
    except:
        return np.nan

def clean_nation(nation_str):
    if pd.isna(nation_str) or nation_str == '':
        return 'Unknown'
    nation = str(nation_str).strip().split()[-1].upper()
    return nation


def clean_position(pos_str):
    """Standardise les positions"""
    if pd.isna(pos_str) or pos_str == '':
        return 'Unknown'
    pos = str(pos_str).strip().upper()
    main_pos = pos.split(',')[0].strip()
    position_mapping = {'GK': 'GK', 'DF': 'DF', 'MF': 'MF', 'FW': 'FW'}
    return position_mapping.get(main_pos, main_pos)

def clean_minutes(min_str):
    """Nettoie les minutes (enlève les virgules: '1,234' -> 1234)"""
    if pd.isna(min_str) or min_str == '':
        return 0
    min_clean = str(min_str).replace(',', '').strip()
    try:
        return int(float(min_clean))
    except:
        return 0

def clean_date(date_str):
    """Standardise les dates au format YYYY-MM-DD"""
    if pd.isna(date_str) or date_str == '':
        return np.nan
    try:
        # Essayer de parser la date
        date_obj = pd.to_datetime(date_str)
        return date_obj.strftime('%Y-%m-%d')
    except:
        return np.nan

def clean_time(time_str):
    """Standardise les heures au format HH:MM"""
    if pd.isna(time_str) or time_str == '':
        return np.nan
    time_clean = str(time_str).strip()
    return time_clean if ':' in time_clean else np.nan

def clean_text(text):
    """Nettoie les colonnes texte"""
    if pd.isna(text) or text == '':
        return 'Unknown'
    return str(text).strip()

# ============================================================================
# TRANSFORMATION DES DONNÉES JOUEURS
# ============================================================================

def transform_players_data(df, team_name):
    """Transforme les données des joueurs d'une équipe"""
    df_clean = df.copy()
    
    # 1. Supprimer les lignes complètement vides
    df_clean = df_clean.dropna(how='all')
    
    # 2. Supprimer les lignes où Player est vide
    df_clean = df_clean[df_clean['Player'].notna()]
    df_clean = df_clean[df_clean['Player'].str.strip() != '']
    
    # 3. Supprimer les doublons
    df_clean = df_clean.drop_duplicates(subset=['Player', 'Team'], keep='first')
    
    # 4. Nettoyer les colonnes texte
    df_clean['Player'] = df_clean['Player'].str.strip()
    df_clean['Team'] = df_clean['Team'].str.strip()
    df_clean['Nation'] = df_clean['Nation'].apply(clean_nation)
    df_clean['Pos'] = df_clean['Pos'].apply(clean_position)
    
    # 5. Nettoyer Age
    df_clean['Age'] = df_clean['Age'].apply(clean_age)
    
    # 6. Nettoyer Minutes
    df_clean['Min'] = df_clean['Min'].apply(clean_minutes)
    
    # 7. Convertir les colonnes numériques
    numeric_columns = ['MP', 'Starts', '90s', 'Gls', 'Ast', 'G+A', 
                      'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR']
    
    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = clean_numeric_column(df_clean[col])
    
    # 8. Ajouter des métadonnées
    df_clean['processing_date'] = datetime.now().strftime('%Y-%m-%d')
    df_clean['season'] = '2024-2025'
    df_clean['league'] = 'Premier League'
    
    # 9. Réorganiser les colonnes
    column_order = [
        'Player', 'Team', 'Nation', 'Pos', 'Age',
        'MP', 'Starts', 'Min', '90s',
        'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt',
        'CrdY', 'CrdR',
        'season', 'league', 'processing_date'
    ]
    
    df_clean = df_clean[[col for col in column_order if col in df_clean.columns]]
    
    return df_clean

# ============================================================================
# TRANSFORMATION DES DONNÉES MATCHS
# ============================================================================

def transform_matches_data(df, team_name):
    """Transforme les données des matchs d'une équipe"""
    df_clean = df.copy()
    
    # 1. Supprimer les lignes vides
    df_clean = df_clean.dropna(how='all')
    
    # 2. Standardiser les dates
    if 'Date' in df_clean.columns:
        df_clean['Date'] = df_clean['Date'].apply(clean_date)
    
    # 3. Standardiser les heures
    if 'Time' in df_clean.columns:
        df_clean['Time'] = df_clean['Time'].apply(clean_time)
    
    # 4. Nettoyer les colonnes texte
    text_columns = ['Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent', 
                    'Captain', 'Formation', 'Opp Formation', 'Referee']
    
    for col in text_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].apply(clean_text)
    
    # 5. Convertir les colonnes numériques
    numeric_columns = ['GF', 'GA', 'xG', 'xGA', 'Poss', 'Attendance']
    
    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = clean_numeric_column(df_clean[col])
    
    # 6. Ajouter l'équipe si pas présent
    if 'Team' not in df_clean.columns:
        df_clean['Team'] = team_name
    
    # 7. Ajouter des métadonnées
    df_clean['processing_date'] = datetime.now().strftime('%Y-%m-%d')
    df_clean['season'] = '2024-2025'
    df_clean['league'] = 'Premier League'
    
    # 8. Réorganiser les colonnes
    column_order = [
        'Team', 'Date', 'Time', 'Comp', 'Round', 'Day', 'Venue',
        'Opponent', 'Result', 'GF', 'GA', 'xG', 'xGA', 'Poss',
        'Attendance', 'Captain', 'Formation', 'Opp Formation', 'Referee',
        'season', 'league', 'processing_date'
    ]
    
    df_clean = df_clean[[col for col in column_order if col in df_clean.columns]]
    
    return df_clean

# ============================================================================
# TRAITEMENT PRINCIPAL
# ============================================================================

def process_all_teams():
    """Traite toutes les équipes du dossier bronze"""
    
    if not os.path.exists(BRONZE_PATH):
        print(f"❌ Le dossier bronze '{BRONZE_PATH}' n'existe pas")
        return
    
    teams = [d for d in os.listdir(BRONZE_PATH) 
            if os.path.isdir(os.path.join(BRONZE_PATH, d))]
    
    print(f"🔄 Traitement de {len(teams)} équipes...\n")
    
    all_players = []
    all_matches = []
    
    stats = {
        'teams': 0,
        'players_initial': 0,
        'players_cleaned': 0,
        'matches_initial': 0,
        'matches_cleaned': 0,
        'errors': []
    }
    
    for team_dir in teams:
        team_path = os.path.join(BRONZE_PATH, team_dir)
        team_name = team_dir.replace('_', ' ')
        
        print(f"📁 Traitement: {team_name}")
        
        # Créer le dossier silver pour l'équipe
        team_silver_dir = os.path.join(SILVER_PATH, team_dir)
        os.makedirs(team_silver_dir, exist_ok=True)
        
        csv_files = [f for f in os.listdir(team_path) if f.endswith('.csv')]
        
        for csv_file in csv_files:
            try:
                file_path = os.path.join(team_path, csv_file)
                df = pd.read_csv(file_path)
                
                initial_count = len(df)
                
                # Déterminer le type de fichier (joueurs ou matchs)
                if 'Player' in df.columns:
                    # Données de joueurs
                    df_clean = transform_players_data(df, team_name)
                    stats['players_initial'] += initial_count
                    stats['players_cleaned'] += len(df_clean)
                    
                    output_file = os.path.join(team_silver_dir, f"{team_dir}_players_clean.csv")
                    df_clean.to_csv(output_file, index=False, encoding='utf-8-sig')
                    all_players.append(df_clean)
                    
                    print(f"  ✅ Joueurs: {initial_count} → {len(df_clean)}")
                    
                elif 'Date' in df.columns and 'Opponent' in df.columns:
                    # Données de matchs
                    df_clean = transform_matches_data(df, team_name)
                    stats['matches_initial'] += initial_count
                    stats['matches_cleaned'] += len(df_clean)
                    
                    output_file = os.path.join(team_silver_dir, f"{team_dir}_matches_clean.csv")
                    df_clean.to_csv(output_file, index=False, encoding='utf-8-sig')
                    all_matches.append(df_clean)
                    
                    print(f"  ✅ Matchs: {initial_count} → {len(df_clean)}")
                    
            except Exception as e:
                error_msg = f"{team_dir}/{csv_file}: {str(e)}"
                stats['errors'].append(error_msg)
                print(f"  ❌ Erreur: {error_msg}")
        
        stats['teams'] += 1
    
    # Créer les fichiers consolidés
    if all_players:
        df_all_players = pd.concat(all_players, ignore_index=True)
        consolidated_players = os.path.join(SILVER_PATH, "all_players_consolidated.csv")
        df_all_players.to_csv(consolidated_players, index=False, encoding='utf-8-sig')
        print(f"\n📊 Fichier consolidé joueurs: {consolidated_players}")
    
    if all_matches:
        df_all_matches = pd.concat(all_matches, ignore_index=True)
        consolidated_matches = os.path.join(SILVER_PATH, "all_matches_consolidated.csv")
        df_all_matches.to_csv(consolidated_matches, index=False, encoding='utf-8-sig')
        print(f"📊 Fichier consolidé matchs: {consolidated_matches}")
    
    # Afficher le résumé
    print("\n" + "="*70)
    print("📈 RÉSUMÉ DU TRAITEMENT")
    print("="*70)
    print(f"Équipes traitées: {stats['teams']}")
    print(f"\nJoueurs:")
    print(f"  - Initial: {stats['players_initial']}")
    print(f"  - Nettoyés: {stats['players_cleaned']}")
    print(f"  - Supprimés: {stats['players_initial'] - stats['players_cleaned']}")
    print(f"\nMatchs:")
    print(f"  - Initial: {stats['matches_initial']}")
    print(f"  - Nettoyés: {stats['matches_cleaned']}")
    print(f"  - Supprimés: {stats['matches_initial'] - stats['matches_cleaned']}")
    
    if stats['errors']:
        print(f"\n⚠️  Erreurs ({len(stats['errors'])}):")
        for error in stats['errors'][:5]:
            print(f"  - {error}")
        if len(stats['errors']) > 5:
            print(f"  ... et {len(stats['errors']) - 5} autres")
    
    print("="*70)

# ============================================================================
# EXÉCUTION
# ============================================================================

if __name__ == "__main__":
    process_all_teams()
    print("\n✅ Transformation terminée!")

🔄 Traitement de 20 équipes...

📁 Traitement: Arsenal
  ✅ Matchs: 58 → 58
  ✅ Joueurs: 38 → 38
📁 Traitement: Aston Villa
  ✅ Matchs: 57 → 57
  ✅ Joueurs: 34 → 34
📁 Traitement: Bournemouth
  ✅ Matchs: 43 → 43
  ✅ Joueurs: 38 → 38
📁 Traitement: Brentford
  ✅ Matchs: 43 → 43
  ✅ Joueurs: 35 → 35
📁 Traitement: Brighton
  ✅ Matchs: 45 → 45
  ✅ Joueurs: 42 → 42
📁 Traitement: Chelsea
  ✅ Matchs: 57 → 57
  ✅ Joueurs: 35 → 35
📁 Traitement: Crystal Palace
  ✅ Matchs: 48 → 48
  ✅ Joueurs: 37 → 37
📁 Traitement: Everton
  ✅ Matchs: 42 → 42
  ✅ Joueurs: 35 → 35
📁 Traitement: Fulham
  ✅ Matchs: 44 → 44
  ✅ Joueurs: 28 → 28
📁 Traitement: Ipswich Town
  ✅ Matchs: 42 → 42
  ✅ Joueurs: 35 → 35
📁 Traitement: Leicester City
  ✅ Matchs: 43 → 43
  ✅ Joueurs: 38 → 38
📁 Traitement: Liverpool
  ✅ Matchs: 56 → 56
  ✅ Joueurs: 29 → 29
📁 Traitement: Manchester City
  ✅ Matchs: 57 → 57
  ✅ Joueurs: 36 → 36
📁 Traitement: Manchester Utd
  ✅ Matchs: 60 → 60
  ✅ Joueurs: 41 → 41
📁 Traitement: Newcastle Utd
  ✅ Matchs: 4